# Notebook de Entrenamiento de Algoritmos para despliegue en BentoML
Este notebook consistirá en la creación y entrenamiento de nuestros de algoritmos aplicados a nuestro dataset **ai4i2020** para predecir/clasificar fallos maquinarios. Intercalando código y explicación, trataremos con diferentes algoritmos de diferente dificultad para contrastar resultados y dejar todos preparados para ser desplegados mediante la API creada en una interfaz gráfica creada mediante **Streamlit**.

Cabe objetar que intentaremos desplegar el código de la mejor forma para su futura reutilización en otras partes del proyecto, siguiendo fomrmato similar al visto en clase.

## Dependencias

In [1]:
import bentoml
import json
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.metrics import accuracy_score

# Algoritmos que probaremos
from sklearn.linear_model import LogisticRegression # fallo/no fallo
from sklearn.ensemble import RandomForestClassifier # clasificación de errores
from sklearn import svm # Support vector machines para clasificación

## 1. Algoritmos de Regresión
Mediante los algoritmos regresivos que probaremos, intentaremos realizar las mejores predicciones posibles para intentar anticiparnos al fallo de nuestras máquinas sintéticas. Ya que la variable que nos interesa (**Machine failure**) tiene un output binario de 0 (no ha habido fallo) o de 1 (ha habido fallo), nos hemos decantado por realizar una regresión logística primero. 

### 1.1 Regresión Logística
Intentaremos predecir el output binario que indica si el proceso ha fallado o no, mediante una implementación básica de este primer algoritmo de regresión.

In [13]:
# Para cada algoritmo haremos una función diferente; creando un modelo diferente para la model store, nombre diferente, ...
def logistic_regression():
    dataset = pd.read_csv("../../data/ai4i2020.csv")

    # Como X cogemos todas las columnas menos la variable que usaremos como 'y' y la variable de identificador
    X, y = dataset.drop(columns=["UDI", "Machine failure"]), dataset["Machine failure"] 

    # Pasamos a númericas las variables categóricas
    columnas_categoricas = X.select_dtypes(include=["object"]).columns
    X = pd.get_dummies(X, columns=columnas_categoricas, drop_first=True)

    # Dividimos el dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)
    
    # Inicializamos el modelo de regresión logística
    lr = LogisticRegression()
    lr.fit(X_train, y_train)
    
    # Realizamos las predicciones
    preds = lr.predict(X_test)

    # Por ahora solo tendremos como métrica de evaluación la accuracy
    accuracy = accuracy_score(y_test, preds)
    print(f"Accuracy obtenida: {accuracy:.4f}")

    # A partir de ahora, configuraremos el modelo para que sea compatible con BentoML
    bento_lr = bentoml.sklearn.save_model(
        "ai4i2020_logistic_regression",
        lr,
        metadata={
            "fecha_entrenamiento": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "dataset": "ai4i2020",
            "framework": "scikit-learn",
            "algoritmo": "regresión logística",
            "precision": accuracy,
            "carta_favorita_cr": "Reina Arquera MOMO SHOW"
        },
    )
    print(f"Modelo de Regresión Logística guardado en la BentoML store como: {bento_lr}")

Y lo ejecutamos corriendo esta simple celda:

In [14]:
logistic_regression()

/home/unai/miniconda3/envs/coran-nlp/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/unai/miniconda3/envs/coran-nlp/lib/python3.11/site-packages/fs/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)  # ty

Accuracy obtenida: 0.9960
Modelo de Regresión Logística guardado en la BentoML store como: Model(tag="ai4i2020_logistic_regression:oioz27wpjsumt7fk")


## 2. Algoritmos de Clasificación
En esta segunda sección de algoritmos, en vez de predecir el resultado del proceso maquinario, intentaremos clasificar el fallo (en el caso de que haya habido) dentro de los distintos tipos de fallas.

### 2.1 Random Forest
Mediante el random forest intentaremos clasificar los errores de los procesos sintéticos en los 5 tipos de errores que existen y enlista nuestro dataset:
- TWF: Fallo por desgaste de herramienta.
- HDF: Fallo por una mala disipación de calor.
- PWF: Falla por potencia fuera de rango.
- OSF: Fallo por un sobreesfuerzo mecánico.
- RNF: Un fallo aleatorio.

In [15]:
def random_forest():
    dataset = pd.read_csv("../../data/ai4i2020.csv")

    # Como X cogemos todas las columnas menos la variable que usaremos como 'y' y la variable de identificador
    X, y = dataset.drop(columns=["UDI", "Machine failure", "TWF", "HDF", "PWF", "RNF", "OSF"]), dataset[["TWF", "HDF", "PWF", "RNF", "OSF"]]

    # Pasamos a númericas las variables categóricas
    columnas_categoricas = X.select_dtypes(include=["object"]).columns
    X = pd.get_dummies(X, columns=columnas_categoricas, drop_first=True)

    # Dividimos el dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

    # Inicializamos el modelo de regresión logística
    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)

    # Realizamos las predicciones
    preds = rf.predict(X_test)

    # Por ahora solo tendremos como métrica de evaluación la accuracy
    accuracy = accuracy_score(y_test, preds)
    print(f"Accuracy obtenida: {accuracy:.4f}")

    # A partir de ahora, configuraremos el modelo para que sea compatible con BentoML
    bento_rf = bentoml.sklearn.save_model(
        "ai4i2020_random_forest",
        rf,
        metadata={
            "fecha_entrenamiento": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "dataset": "ai4i2020",
            "framework": "scikit-learn",
            "algoritmo": "clasificación con random forest",
            "precision": accuracy,
            "carta_favorita_cr": "Reina Arquera MOMO SHOW"
        },
    )
    print(f"Modelo de Clasificación con Random Forest guardado en la BentoML store como: {bento_rf}")

Para ejecutar el Random Forest:

In [16]:
random_forest()

Accuracy obtenida: 0.9710
Modelo de Clasificación con Random Forest guardado en la BentoML store como: Model(tag="ai4i2020_random_forest:rzbs6agpjsumt7fk")


### 2.2 Suppor Vector Machines (SVMs)
Este algoritmo de clasificación más pesado buscará separar las observaciones donde haya habido fallo y donde no en un espacio mediante el empleo de vectores clasificadores. Es decir, volveremos a emplear como variable 'y' la variable de **Machine Failure**.

In [17]:
def support_vector_machines():
    dataset = pd.read_csv("../../data/ai4i2020.csv")

    # Como X cogemos todas las columnas menos la variable que usaremos como 'y' y la variable de identificador
    X, y = dataset.drop(columns=["UDI", "Machine failure"]), dataset["Machine failure"] 

    # Pasamos a númericas las variables categóricas
    columnas_categoricas = X.select_dtypes(include=["object"]).columns
    X = pd.get_dummies(X, columns=columnas_categoricas, drop_first=True)

    # Dividimos el dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

    model_svm = svm.SVC()
    model_svm.fit(X_train, y_train)

    # Realizamos las predicciones
    preds = model_svm.predict(X_test)

    # Por ahora solo tendremos como métrica de evaluación la accuracy
    accuracy = accuracy_score(y_test, preds)
    print(f"Accuracy obtenida: {accuracy:.4f}")

    # A partir de ahora, configuraremos el modelo para que sea compatible con BentoML
    bento_svm = bentoml.sklearn.save_model(
        "ai4i2020_suppor_vector_machine",
        model_svm,
        metadata={
            "fecha_entrenamiento": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "dataset": "ai4i2020",
            "framework": "scikit-learn",
            "algoritmo": "clasificación con svms",
            "precision": accuracy,
            "carta_favorita_cr": "Reina Arquera MOMO SHOW"
        },
    )
    print(f"Modelo de Clasificación con Support Vector Machines (SVMs) guardado en la BentoML store como: {bento_svm}")

Ejecutémoslo:

In [18]:
support_vector_machines()

Accuracy obtenida: 0.9695
Modelo de Clasificación con Support Vector Machines (SVMs) guardado en la BentoML store como: Model(tag="ai4i2020_suppor_vector_machine:y3io4dgpjsumt7fk")


## Para alistar los modelos creados en la BentoML Store:
Al ejecutar el siguiente comando, podremos ver datos sobre el modelo como su nombre (con su tag), el módulo usado, el tamaño final del modelo y la fecha de creación del modelo. Cada vez que ejecutemos una función que llame al entrenamiento de un algoritmo, se creará otro con un tag distinto. Pudiendo haber más de una instancia por algoritmo, el tag los diferenciará. En el caso de que queramos eliminar una instancia, tendremos que usar el siguiente comando:

```bash
bendoml models delete <nombre del modelo (con su tag)>
```

In [22]:
!bentoml models list

/home/unai/miniconda3/envs/coran-nlp/lib/python3.11/site-packages/fs/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)  # type: ignore
 Tag                           Module           Size        Creation Time       
 ai4i2020_suppor_vector_mach…  bentoml.sklearn  43.36 MiB   2025-12-02 08:01:50 
 ai4i2020_random_forest:rzbs…  bentoml.sklearn  22.91 MiB   2025-12-02 08:00:15 
 ai4i2020_logistic_regressio…  bentoml.sklearn  275.34 KiB  2025-12-02 07:59:28 
 make_classification_logisti…  bentoml.sklearn  1.35 KiB    2025-11-28 19:59:51 
 flower_model_iris:mrlpxugmq…  bentoml.sklearn  182.98 KiB  2025-11-28 19:23:52 
